In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-100rfq-30cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

7690

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

bid_count = 0
bid_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        bid_count += 1   


print(f"#BIDs =  {bid_count}")

#BIDs =  2496


In [18]:
df_test = grouped_data.get_group('001a846fea46ae15354512921ab5041d299351e7dd6eb207eeabbcf794356731')
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
42390,received_tx,108,BID,001a846fea46ae15354512921ab5041d299351e7dd6eb2...,None,1.669864e+09
42391,before_tendermint,119,BID,001a846fea46ae15354512921ab5041d299351e7dd6eb2...,None,1.669864e+09
42401,check_tx,616,BID,001a846fea46ae15354512921ab5041d299351e7dd6eb2...,None,1.669864e+09
42477,check_tx,2069,BID,001a846fea46ae15354512921ab5041d299351e7dd6eb2...,None,1.669864e+09
42481,deliver_tx,2137,BID,001a846fea46ae15354512921ab5041d299351e7dd6eb2...,None,1.669864e+09
42482,end_block,2139,BID,001a846fea46ae15354512921ab5041d299351e7dd6eb2...,None,1.669864e+09
42483,commit_tx,2181,BID,001a846fea46ae15354512921ab5041d299351e7dd6eb2...,None,1.669864e+09


In [6]:
time_all_committed_bids = {}
time_passed_bids = []
validated_time = []
accepted_time = []
committed_time = []

bid_underfit = []
bid_underfit_dict = {}

bid_overfit = []
bid_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            bid_underfit.append(temp_list)
            bid_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            #time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            bid_overfit.append(actual_time)
            bid_overfit_dict[tx_id[i]] = temp_list

In [7]:
df = grouped_data.get_group("3066cad3ab9009c84499c002f3b150864fc0b5af46bd4cb1ab5407ab2396a336")
df[df['Event'] == 'received_tx']['Time'].to_list()[0]

106

In [8]:
df[df['Event'] == 'received_tx']

,Event,Time,Tx_type,Tx_id,Accept_id,time
40977,received_tx,106,BID,3066cad3ab9009c84499c002f3b150864fc0b5af46bd4c...,None,1.669864e+09


In [9]:
bid_overfit_dict

{'001a846fea46ae15354512921ab5041d299351e7dd6eb207eeabbcf794356731': [108,
  119,
  616,
  2069,
  2137,
  2139,
  2181],
 '0082f6f58d2b9dbbdf09166ae51cb463eda825fe79660c69f88e2af8858d0785': [109,
  119,
  689,
  775,
  838,
  840,
  882],
 '03abcea500b8a5006d19c04326809cff957cb937a84f914e1e00122ecfade2f0': [106,
  118,
  230,
  492,
  552,
  554,
  597],
 '052525f238cdd302ee75ae4bf0df22165bc41dcfa95721338ca3ce639ae0c60b': [107,
  117,
  694,
  2154,
  3705,
  3709,
  3720],
 '0542590e478e8b6d41983421c8511cd9b230d23fcc190df92c3b4b782b484384': [105,
  116,
  290,
  861,
  1362,
  1488,
  1496],
 '0a38adfe986e9e6a45da868a37439c68136e54caa1c33dce947629363b8073a9': [105,
  115,
  565,
  622,
  686,
  687,
  730],
 '0ad467e734e0af06839cf3e50cbb9a6c78b45ee8385173e5d320928f2a89c7e5': [107,
  117,
  1088,
  1146,
  1209,
  1211,
  1254],
 '0d1cb4d04c575a9ceef6c0e57ba96745271cc50fd1834a1c0a518573602be701': [104,
  114,
  986,
  2268,
  2339,
  2347,
  2384],
 '0ef026f08564fcb73a5487f9fc1e34935e

In [10]:
len(time_passed_bids)

2365

In [11]:
len(bid_overfit)

130

In [12]:
stat.mean(time_passed_bids)

98.87653276955602

In [13]:
stat.mean(bid_overfit)

1519.8384615384616

In [14]:
# Analysis of CREATE Transaction
bid_mean = "{:.2f}".format(stat.mean(time_passed_bids))
bid_median = stat.median(time_passed_bids)
bid_max = max(time_passed_bids)
bid_min = min(time_passed_bids)
bid_std_dev = "{:.2f}".format(stat.stdev(time_passed_bids))

print(bid_mean)
print(bid_median)
print(bid_max)
print(bid_min)
print(bid_std_dev)
print(f"total bids = {len(time_passed_bids)}")

bid_dict = {
               'min': bid_min,
               'max': bid_max,
               'average': bid_mean,
               'median' : bid_median,
               'standard_deviation' : bid_std_dev,
               '# of txs' : len(time_passed_bids)}

98.88
85
791
75
46.27
total bids = 2365


In [15]:
results = pd.DataFrame(bid_dict, index = ['BID'])
results.to_csv('bid_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
BID,75,791,98.88,85,46.27,2365


In [16]:
per_event_dict = {
                    'validated': validated_time,
                    'accepted' : accepted_time,
                    'committed': committed_time
}
per_event_df = pd.DataFrame(per_event_dict)
per_event_df.to_csv('bid_per_event.csv', sep=',', encoding='utf-8')

In [17]:
time_all_committed_bids

{'000618f6c2f34b4685c2db8d27aef53c345bddc25e716d635da8b7e32e3ddca9': 81,
 '0015985b832ef4e86b84a5397a16c6357225e4cc459a3e6c8a46988a7b555b97': 82,
 '001a846fea46ae15354512921ab5041d299351e7dd6eb207eeabbcf794356731': 2073,
 '0036761f1f3c8de5fc7fd29e24dabeac1f99e7f7433eac9393cf3a92583cafa3': 85,
 '00450cb9ea7dd2f513ce57e137ab433dd9655ec4b0bc5c15ada3e4b1937afd13': 86,
 '0057f28b3d0d9b9aa8a9795906e4b957697e05dbcf14660d2ead1d524feb473b': 84,
 '005961c5aaf3f2f3d7fe93e252b419d2f3ea23d2e6f76aae5fcdc7f8a906119f': 115,
 '00623933577a88c2ab2bf88d91a3bb2609adb48026cacdaf93ad6504758a68f1': 84,
 '0082f6f58d2b9dbbdf09166ae51cb463eda825fe79660c69f88e2af8858d0785': 773,
 '00a3c468271c33c7b12186ff4551384e9e28647c7f1a1b8accbde57edf677dde': 84,
 '00b2e05e9782573bdba9f65d13a746da9e2712da7121f9403ed12005144a31d7': 84,
 '00b3e98f68f510f893ca97304c947c5d471b96d4c2ef7877ebf483fcf87c1949': 82,
 '00b5064932135c47bde7f7a6d052fb8c66713982a29e41af856859969091bca1': 83,
 '00b6fa17d30313a53c32720e5a6c31816f1ea5d505ad5